# Server VITS on Trition Docker
- 선수 노트북: 아래를 먼저 실행해야 합니다
    - 0.0-create-tts-vits-model.ipynb

## 1.  Setup environment
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [18]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append(os.path.abspath("./vits"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Run Docker


#### (1) 터미널 실행
#### (2) SageMaker Triton 도커 컨테이너 실행 -- triton 24.05 version :
* ECR에서 TRITON 서버 도커 이미지를 가져오는 과정 1/ECR 로그인(get-loginp-password) 2/도커이미지 가져오기(pull) 
* 3/TRITON서버를 자세한 로깅 옵션과 함께 실행하기 (docker run \)
```
# Move to current folder (e.g.: /home/ec2-user/SageMaker/lab/00-trition-tts-vits/ )
docker run --gpus=1 --rm -p8000:8000 -p8001:8001 -p8002:8002 -v `pwd`/triton-serve-jit:/models 763104351884.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tritonserver:24.05-py3 tritonserver --model-repository=/models --log-verbose=3 --log-info=1 --log-warning=1 --log-error=1
```

# 3. Run single inference on Triton Client

In [19]:
from local_utils.triton_util import setup_triton_client
triton_client, grpcclient = setup_triton_client()

In [20]:
import utils #VITS 유틸리티 함수
# from local_utils.triton_util import  create_input_data, create_client_payload
from local_utils.triton_util import infer_triton_client, create_input_payload, create_input_payload_padding

import numpy as np
import tritonclient.grpc as grpcclient
import time

model_name = "tts-vits" #모델 이름
# Load Hyper-parameter
hps = utils.get_hparams_from_file("vits/configs/ljs_base.json")

# Define text to be spoken


# text = "Claude is AI for all of us. Whether you're brainstorming alone or building with a team of thousands, Claude is here to help"
text = "Today, we're announcing the Claude 3 model family, which sets new industry benchmarks across a wide range of cognitive tasks. The family includes three state-of-the-art models in ascending order of capability: Claude 3 Haiku, Claude 3 Sonnet, and Claude 3 Opus. Each successive model offers increasingly powerful performance, allowing users to select the optimal balance of intelligence, speed, and cost for their specific application."
inputs = create_input_payload_padding(text, hps, grpcclient)
    
# Define output
outputs = []
# outputs.append(grpcclient.InferRequestedOutput('OUTPUT_0'))
outputs.append(grpcclient.InferRequestedOutput('OUTPUT_0'))

print(inputs)

# infernece
start_time = time.perf_counter()
result = infer_triton_client(triton_client, model_name, inputs, outputs)
end_time = time.perf_counter()
latency = (end_time - start_time) * 1000  # ms 단위로 변환
print("latency: ", latency)

# Show output shape
output0_data = result.as_numpy('OUTPUT_0')
output0_data.shape


x_np length 965
padding_length:  1083
x_np length after padding 2048
x data shape: (1, 2048)
x_length shape: (1, 1)
noise_scale shape: (1, 1)
length_scale shape: (1, 1)
noise_scale_w shape: (1, 1)
[<tritonclient.grpc._infer_input.InferInput object at 0x7fe3345fae00>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe3342db250>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe3305be9b0>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe3305bc820>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe3305beaa0>]
latency:  1089.2156040063128


(1, 1, 798208)

## Play Audio

In [21]:
import IPython.display as ipd #오디오 재생용

audio = output0_data[0,0]
audio

ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

# Benchmarking with variable input

In [22]:
text = "Today, we're announcing the Claude 3 model family, which sets new industry benchmarks across a wide range of cognitive tasks. The family includes three state-of-the-art models in ascending order of capability: Claude 3 Haiku, Claude 3 Sonnet, and Claude 3 Opus. Each successive model offers increasingly powerful performance, allowing users to select the optimal balance of intelligence, speed, and cost for their specific application."
inputs0 = create_input_payload(text, hps, grpcclient)
text = "Perplexity says these ads will not influence search results and the company remains committed to providing unbiased answers to queries"
inputs1 = create_input_payload(text, hps, grpcclient)
text = "The company assures users that these ads will not impact the impartiality of search results, and that these sponsored questions are clearly marked"
inputs2 = create_input_payload(text, hps, grpcclient)


# Define output
outputs = []
outputs.append(grpcclient.InferRequestedOutput('OUTPUT_0'))



x data shape: (1, 965)
x_length shape: (1, 1)
noise_scale shape: (1, 1)
length_scale shape: (1, 1)
noise_scale_w shape: (1, 1)
x data shape: (1, 281)
x_length shape: (1, 1)
noise_scale shape: (1, 1)
length_scale shape: (1, 1)
noise_scale_w shape: (1, 1)
x data shape: (1, 305)
x_length shape: (1, 1)
noise_scale shape: (1, 1)
length_scale shape: (1, 1)
noise_scale_w shape: (1, 1)


In [24]:
# 여러 입력 세트 준비
inputs_list = [
    inputs0,  # 첫 번째 입력 세트
    inputs1,  # 두 번째 입력 세트
    inputs2   # 세 번째 입력 세트
]

# 벤치마크 실행
stats = run_benchmark_example(triton_client, model_name, inputs_list, outputs)

워밍업 실행 중...
벤치마크 시작: 64개 요청 처리

=== 전체 벤치마크 결과 ===
총 요청 수: 64
총 처리 시간 (초): 18.55
처리량 (requests/sec): 3.45
평균 지연시간 (ms): 2269.87
최소 지연시간 (ms): 595.88
최대 지연시간 (ms): 13240.07
중간값 지연시간 (ms): 766.54
표준편차 (ms): 3743.39
P90 지연시간 (ms): 8832.15
P95 지연시간 (ms): 12850.32
P99 지연시간 (ms): 13149.81

=== 입력별 벤치마크 결과 ===

Input [<tritonclient.grpc._infer_input.InferInput object at 0x7fe3345f99c0>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe3305bf880>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe3305bcf40>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe3305bf850>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe3305bc2e0>]:
  요청 수: 22
  비율 (%): 34.38
  평균 지연시간 (ms): 3335.65
  최소 지연시간 (ms): 665.32
  최대 지연시간 (ms): 13240.07
  P95 지연시간 (ms): 13085.60

Input [<tritonclient.grpc._infer_input.InferInput object at 0x7fe3305be500>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe3305bfd60>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe3305b

# Benchmarking with Padding

In [25]:
from local_utils.triton_util import run_benchmark_example, create_input_payload_padding

In [26]:
text = "Today, we're announcing the Claude 3 model family, which sets new industry benchmarks across a wide range of cognitive tasks. The family includes three state-of-the-art models in ascending order of capability: Claude 3 Haiku, Claude 3 Sonnet, and Claude 3 Opus. Each successive model offers increasingly powerful performance, allowing users to select the optimal balance of intelligence, speed, and cost for their specific application."
inputs0 = create_input_payload_padding(text, hps, grpcclient)
text = "Perplexity says these ads will not influence search results and the company remains committed to providing unbiased answers to queries"
inputs1 = create_input_payload_padding(text, hps, grpcclient)
text = "The company assures users that these ads will not impact the impartiality of search results, and that these sponsored questions are clearly marked"
inputs2 = create_input_payload_padding(text, hps, grpcclient)


# Define output
outputs = []
outputs.append(grpcclient.InferRequestedOutput('OUTPUT_0'))



x_np length 965
padding_length:  1083
x_np length after padding 2048
x data shape: (1, 2048)
x_length shape: (1, 1)
noise_scale shape: (1, 1)
length_scale shape: (1, 1)
noise_scale_w shape: (1, 1)
x_np length 281
padding_length:  1767
x_np length after padding 2048
x data shape: (1, 2048)
x_length shape: (1, 1)
noise_scale shape: (1, 1)
length_scale shape: (1, 1)
noise_scale_w shape: (1, 1)
x_np length 305
padding_length:  1743
x_np length after padding 2048
x data shape: (1, 2048)
x_length shape: (1, 1)
noise_scale shape: (1, 1)
length_scale shape: (1, 1)
noise_scale_w shape: (1, 1)


In [27]:
# 여러 입력 세트 준비
inputs_list = [
    inputs0,  # 첫 번째 입력 세트
    inputs1,  # 두 번째 입력 세트
    inputs2   # 세 번째 입력 세트
]

# 벤치마크 실행
stats = run_benchmark_example(triton_client, model_name, inputs_list, outputs)

워밍업 실행 중...
벤치마크 시작: 64개 요청 처리

=== 전체 벤치마크 결과 ===
총 요청 수: 64
총 처리 시간 (초): 11.77
처리량 (requests/sec): 5.44
평균 지연시간 (ms): 1393.13
최소 지연시간 (ms): 170.36
최대 지연시간 (ms): 1604.39
중간값 지연시간 (ms): 1464.19
표준편차 (ms): 268.32
P90 지연시간 (ms): 1535.65
P95 지연시간 (ms): 1559.52
P99 지연시간 (ms): 1588.53

=== 입력별 벤치마크 결과 ===

Input [<tritonclient.grpc._infer_input.InferInput object at 0x7fe33049fb20>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe3304609a0>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe330461270>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe330461060>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe330460ee0>]:
  요청 수: 25
  비율 (%): 39.06
  평균 지연시간 (ms): 1333.69
  최소 지연시간 (ms): 170.36
  최대 지연시간 (ms): 1561.50
  P95 지연시간 (ms): 1520.43

Input [<tritonclient.grpc._infer_input.InferInput object at 0x7fe3314ab820>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe3344be7a0>, <tritonclient.grpc._infer_input.InferInput object at 0x7fe3305eab60>